# Registry

> Unified plugin registry for managing multiple domain-specific plugin systems with configuration persistence

In [ ]:
#| default_exp core.registry

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import json
from dataclasses import dataclass, field
from pathlib import Path
from typing import Dict, List, Optional, Any, TypeVar, Type

from cjm_fasthtml_jsonschema.core.dataclass import dataclass_to_jsonschema

## PluginMetadata

Metadata describing a plugin for display and configuration management.

In [ ]:
#| export
@dataclass
class PluginMetadata:
    """Metadata describing a plugin for display and configuration management."""
    name: str  # Internal plugin identifier
    category: str  # Plugin category string (application-defined)
    title: str  # Display title for the plugin
    config_schema: Dict[str, Any]  # JSON Schema for plugin configuration
    config_dataclass: Optional[Type] = None  # Configuration dataclass type (if available)
    description: Optional[str] = None  # Plugin description
    version: Optional[str] = None  # Plugin version
    is_configured: bool = False  # Whether the plugin has saved configuration
    
    def get_unique_id(self) -> str:  # String in format 'category_name'
        """Generate unique ID for this plugin."""
        return f"{self.category}_{self.name}"

## Unified Plugin Registry

The `UnifiedPluginRegistry` manages multiple domain-specific plugin systems (transcription, LLM, image generation, etc.) in a single application. It provides:

- String-based categories (no hardcoded enums)
- Configuration persistence (JSON files)
- Integration with FastHTML settings and resource management
- Support for local and cloud plugins

This registry works with any plugin system built on `cjm-plugin-system`.

In [ ]:
#| export
T = TypeVar('T')

class UnifiedPluginRegistry:
    """Unified registry for multiple domain-specific plugin systems with configuration persistence."""
    
    def __init__(self, 
                 config_dir: Optional[Path] = None  # Directory for plugin configuration files (default: 'configs')
                ):
        """Initialize the unified plugin registry."""
        self._managers: Dict[str, Any] = {}  # category -> manager
        self._categories: Dict[str, str] = {}  # category -> display_name
        self._plugins: Dict[str, PluginMetadata] = {}  # unique_id -> metadata
        self._config_dir = config_dir or Path("configs")
        self._config_dir.mkdir(exist_ok=True, parents=True)
    
    def register_plugin_manager(
        self,
        category: str,  # Category name (e.g., "transcription", "llm")
        manager: Any,  # Domain-specific plugin manager
        display_name: Optional[str] = None,  # Display name for UI
        auto_discover: bool = True  # Automatically discover plugins?
    ) -> List[PluginMetadata]:  # List of discovered plugin metadata
        """Register a domain-specific plugin manager."""
        self._managers[category] = manager
        self._categories[category] = display_name or category.title()
        
        if auto_discover:
            return self._discover_and_register_plugins(category, manager)
        return []
    
    def register_plugin_system(
        self,
        category: str,  # Category name (e.g., "transcription", "llm")
        plugin_interface: Type,  # Plugin interface class (e.g., TranscriptionPlugin)
        display_name: Optional[str] = None,  # Display name for UI
        auto_discover: bool = True  # Automatically discover plugins?
    ) -> List[PluginMetadata]:  # List of discovered plugin metadata
        """
        Create and register a plugin system in one step. This is a convenience method that creates a PluginManager with the
        specified interface and registers it with the registry.
        """
        from cjm_plugin_system.core.manager import PluginManager
        
        manager = PluginManager(plugin_interface=plugin_interface)
        return self.register_plugin_manager(
            category=category,
            manager=manager,
            display_name=display_name,
            auto_discover=auto_discover
        )
    
    def _discover_and_register_plugins(
        self,
        category: str,  # Category name
        manager: Any  # Plugin manager instance
    ) -> List[PluginMetadata]:  # List of discovered plugin metadata
        """Discover plugins from manager and register their metadata."""
        discovered = manager.discover_plugins()
        plugin_metadatas = []
        
        for plugin_data in discovered:
            # Get config dataclass type from manager and convert to JSON schema
            config_dataclass = manager.get_plugin_config_dataclass(plugin_data.name)
            
            config_schema = dataclass_to_jsonschema(config_dataclass)
            
            # Create plugin metadata
            metadata = PluginMetadata(
                name=plugin_data.name,
                category=category,
                title=config_schema.get('title', plugin_data.name),
                config_schema=config_schema,
                config_dataclass=config_dataclass,
                version=plugin_data.version,
                description=config_schema.get('description')
            )
            
            # Check if plugin is configured
            config_file = self._config_dir / f"{metadata.get_unique_id()}.json"
            metadata.is_configured = config_file.exists()
            
            # Store metadata
            self._plugins[metadata.get_unique_id()] = metadata
            plugin_metadatas.append(metadata)
        
        return plugin_metadatas
    
    def get_manager(
        self,
        category: str,  # Category name
        manager_type: Optional[Type[T]] = None  # Optional type hint for IDE autocomplete
    ) -> Optional[T]:  # Plugin manager instance
        """Get plugin manager for a specific category."""
        return self._managers.get(category)
    
    def get_categories(self) -> List[str]:  # Sorted list of category names
        """Get all registered categories."""
        return sorted(self._categories.keys())
    
    def get_category_display_name(self, 
                                   category: str  # Category name
                                  ) -> str:  # Display name or category name if not set
        """Get display name for a category."""
        return self._categories.get(category, category.title())
    
    def get_plugin(self, 
                   unique_id: str  # Plugin unique identifier (format: 'category_name')
                  ) -> Optional[PluginMetadata]:  # Plugin metadata if found, None otherwise
        """Get plugin metadata by unique ID."""
        return self._plugins.get(unique_id)
    
    def get_plugins_by_category(self, 
                                category: str  # Category name
                               ) -> List[PluginMetadata]:  # List of plugin metadata for the category
        """Get all plugins in a category."""
        return [p for p in self._plugins.values() if p.category == category]
    
    def get_all_plugins(self) -> List[PluginMetadata]:  # List of all plugin metadata
        """Get all plugins across all categories."""
        return list(self._plugins.values())
    
    def get_categories_with_plugins(self) -> List[str]:  # Sorted list of categories with plugins
        """Get categories that have registered plugins."""
        categories = set(p.category for p in self._plugins.values())
        return sorted(categories)
    
    def load_plugin_config(self, 
                          unique_id: str  # Plugin unique identifier
                         ) -> Dict[str, Any]:  # Configuration dictionary (empty if no config exists)
        """Load saved configuration for a plugin."""
        config_file = self._config_dir / f"{unique_id}.json"
        if config_file.exists():
            with open(config_file, 'r') as f:
                return json.load(f)
        return {}
    
    def save_plugin_config(self, 
                          unique_id: str,  # Plugin unique identifier
                          config: Dict[str, Any]  # Configuration dictionary to save
                         ) -> bool:  # True if save succeeded, False otherwise
        """Save configuration for a plugin."""
        try:
            config_file = self._config_dir / f"{unique_id}.json"
            with open(config_file, 'w') as f:
                json.dump(config, f, indent=2)
            
            # Update plugin's configured status
            if unique_id in self._plugins:
                self._plugins[unique_id].is_configured = True
            
            return True
        except Exception as e:
            print(f"Error saving config for {unique_id}: {e}")
            return False
    
    def delete_plugin_config(self, 
                            unique_id: str  # Plugin unique identifier
                           ) -> bool:  # True if deletion succeeded, False otherwise
        """Delete saved configuration for a plugin."""
        try:
            config_file = self._config_dir / f"{unique_id}.json"
            if config_file.exists():
                config_file.unlink()
            
            # Update plugin's configured status
            if unique_id in self._plugins:
                self._plugins[unique_id].is_configured = False
            
            return True
        except Exception as e:
            print(f"Error deleting config for {unique_id}: {e}")
            return False

## Example Usage

In [ ]:
# Create a mock plugin manager for testing
from dataclasses import dataclass, field
from cjm_plugin_system.utils.validation import SCHEMA_TITLE, SCHEMA_DESC, SCHEMA_ENUM

@dataclass
class MockPluginData:
    name: str
    version: str

@dataclass
class WhisperTinyConfig:
    """Configuration for Whisper Tiny model."""
    __schema_name__ = "whisper_tiny"
    __schema_title__ = "Whisper Tiny Configuration"
    __schema_description__ = "Configuration for whisper_tiny"
    
    device: str = field(default="cpu", metadata={SCHEMA_ENUM: ["cpu", "cuda"]})

@dataclass
class WhisperBaseConfig:
    """Configuration for Whisper Base model."""
    __schema_name__ = "whisper_base"
    __schema_title__ = "Whisper Base Configuration"
    __schema_description__ = "Configuration for whisper_base"
    
    device: str = field(default="cpu", metadata={SCHEMA_ENUM: ["cpu", "cuda"]})

class MockPluginManager:
    """Mock plugin manager for testing."""
    
    def __init__(self):
        self._config_dataclasses = {
            "whisper_tiny": WhisperTinyConfig,
            "whisper_base": WhisperBaseConfig
        }
    
    def discover_plugins(self):
        return [
            MockPluginData("whisper_tiny", "1.0.0"),
            MockPluginData("whisper_base", "1.0.0")
        ]

    def get_plugin_config_dataclass(self, name:str):
        return self._config_dataclasses.get(name)

In [ ]:
# Example: Create registry and register a plugin manager
import tempfile

with tempfile.TemporaryDirectory() as tmpdir:
    registry = UnifiedPluginRegistry(config_dir=Path(tmpdir))
    
    # Register transcription plugin manager
    mock_manager = MockPluginManager()
    discovered = registry.register_plugin_manager(
        category="transcription",
        manager=mock_manager,
        display_name="Transcription Plugins"
    )
    
    print(f"Discovered {len(discovered)} plugins")
    for plugin in discovered:
        print(f"  - {plugin.title} (v{plugin.version})")

Discovered 2 plugins
  - Whisper Tiny Configuration (v1.0.0)
  - Whisper Base Configuration (v1.0.0)


In [ ]:
# Example: Query registry
with tempfile.TemporaryDirectory() as tmpdir:
    registry = UnifiedPluginRegistry(config_dir=Path(tmpdir))
    registry.register_plugin_manager("transcription", MockPluginManager())
    
    # Get categories
    print(f"Categories: {registry.get_categories()}")
    print(f"Categories with plugins: {registry.get_categories_with_plugins()}")
    
    # Get plugins
    plugins = registry.get_plugins_by_category("transcription")
    print(f"\nTranscription plugins: {len(plugins)}")
    for plugin in plugins:
        print(f"  - {plugin.name} ({plugin.get_unique_id()})")
    
    # Get specific plugin
    plugin = registry.get_plugin("transcription_whisper_tiny")
    print(f"\nPlugin metadata:")
    print(f"  Title: {plugin.title}")
    print(f"  Category: {plugin.category}")
    print(f"  Is configured: {plugin.is_configured}")

Categories: ['transcription']
Categories with plugins: ['transcription']

Transcription plugins: 2
  - whisper_tiny (transcription_whisper_tiny)
  - whisper_base (transcription_whisper_base)

Plugin metadata:
  Title: Whisper Tiny Configuration
  Category: transcription
  Is configured: False


In [ ]:
# Example: Save and load plugin configuration
with tempfile.TemporaryDirectory() as tmpdir:
    registry = UnifiedPluginRegistry(config_dir=Path(tmpdir))
    registry.register_plugin_manager("transcription", MockPluginManager())
    
    unique_id = "transcription_whisper_tiny"
    
    # Check initial state
    plugin = registry.get_plugin(unique_id)
    print(f"Initially configured: {plugin.is_configured}")
    
    # Save configuration
    config = {"device": "cuda", "model_size": "tiny"}
    success = registry.save_plugin_config(unique_id, config)
    print(f"Save succeeded: {success}")
    
    # Check updated state
    plugin = registry.get_plugin(unique_id)
    print(f"Now configured: {plugin.is_configured}")
    
    # Load configuration
    loaded_config = registry.load_plugin_config(unique_id)
    print(f"Loaded config: {loaded_config}")
    
    # Delete configuration
    success = registry.delete_plugin_config(unique_id)
    print(f"Delete succeeded: {success}")
    
    # Check final state
    plugin = registry.get_plugin(unique_id)
    print(f"Finally configured: {plugin.is_configured}")

Initially configured: False
Save succeeded: True
Now configured: True
Loaded config: {'device': 'cuda', 'model_size': 'tiny'}
Delete succeeded: True
Finally configured: False


In [ ]:
# Example: Register multiple plugin managers (multi-domain)
@dataclass
class LlamaConfig:
    """Configuration for Llama model."""
    __schema_name__ = "llama3"
    __schema_title__ = "Llama3 Config"
    __schema_description__ = "Configuration for llama3"

@dataclass
class GPT4Config:
    """Configuration for GPT-4 model."""
    __schema_name__ = "gpt4"
    __schema_title__ = "GPT4 Config"
    __schema_description__ = "Configuration for gpt4"

class MockLLMManager:
    def __init__(self):
        self._config_dataclasses = {
            "llama3": LlamaConfig,
            "gpt4": GPT4Config
        }
    
    def discover_plugins(self):
        return [MockPluginData("llama3", "1.0.0"), MockPluginData("gpt4", "1.0.0")]
    
    def get_plugin_config_dataclass(self, name: str):
        return self._config_dataclasses.get(name)

with tempfile.TemporaryDirectory() as tmpdir:
    registry = UnifiedPluginRegistry(config_dir=Path(tmpdir))
    
    # Register multiple domains
    registry.register_plugin_manager("transcription", MockPluginManager())
    registry.register_plugin_manager("llm", MockLLMManager(), display_name="Language Models")
    
    # Show all categories and plugins
    print("Categories:")
    for category in registry.get_categories():
        display = registry.get_category_display_name(category)
        plugins = registry.get_plugins_by_category(category)
        print(f"  {display} ({category}): {len(plugins)} plugins")
    
    # Show all plugins
    print(f"\nTotal plugins: {len(registry.get_all_plugins())}")
    for plugin in registry.get_all_plugins():
        print(f"  - [{plugin.category}] {plugin.name}")

Categories:
  Language Models (llm): 2 plugins
  Transcription (transcription): 2 plugins

Total plugins: 4
  - [transcription] whisper_tiny
  - [transcription] whisper_base
  - [llm] llama3
  - [llm] gpt4


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()